# Documentación Proyecto final: programa de creación de horarios de forma eficiente

15/11/2023

**Primero lo que hice fue** la creación de clases y estructuras con sus atributos para cada uno de los objetos que ibamos a guardar, en este caso, profesores, aulas, cursos, secciones y los horarios.

In [ ]:
import networkx as nx
import json
import matplotlib.pyplot as plt

#profesores
class Profesor:
    def __init__(self, id_profesor, nombre, horarios_disp):
        self.id = id_profesor
        self.nombre = nombre
        self.horarios_disp = horarios_disp

#cursos
class Curso:
    def __init__(self, id_curso, nombre, horas, lab, profe_disp, capacidad):
        self.id = id_curso
        self.nombre = nombre
        self.horas = horas
        self.lab = lab
        self.profe_disp = profe_disp
        self.capacidad = capacidad

#Aulas
class Aula:
    def __init__(self, numero, edificio, lab, capacidad, horarios_disp):
        self.numero = numero
        self.edificio = edificio
        self.lab = lab
        self.capacidad = capacidad
        self.horarios_disp = horarios_disp

#secciones
class Seccion:
    def __init__(self, id_seccion, profesor_asignado, horario, dias, curso):
        self.id = id_seccion
        self.profesor_asignado = profesor_asignado
        self.horario = horario
        self.dias = dias
        self.curso = curso

# Cargar datos JSON
with open('Datos.json', 'r', encoding='utf-8') as file:
    datos = json.load(file)

Después de buscar e investigar los algoritmos que me podrían funcionar, decidí utilizar un grafo bipartito que conectara los profesores con las clases o materias que podian impartir, esto ya que así podía vincularlos sin estar conectados entre sí cada conjunto, esto para que tambien pudiera iterar en cada conjunto y hacer busquedas o editar datos de cada elemento de forma más sencilla. Para esto utilicé dos librerías, una sería matplotlib y la otra networkx, la primera se encarga de graficarlo y la segunda me ayuda a agreegar nodos, arista y editar el grafo de forma más sencilla

In [ ]:
#función para generar grafo bipartito
def generar_grafo_bipartito(datos):
    grafo = nx.Graph()
    
    #conjunto de datos 'Cursos'
    cursos = datos["Datos"][2]["Curso"]
    #se crea un nodo por cada elemento del conjunto
    for curso in cursos:
        id_curso = curso["ID"]
        grafo.add_node(id_curso, bipartite=0, tipo="Curso")

    # Conjunto de datos 'Profesores'
    profesores = datos["Datos"][0]["Profesor"]
    #se crea un nodo por cada elemento del conjunto
    for profesor in profesores:
        nombre_profesor = profesor['Nombre']
        grafo.add_node(nombre_profesor, bipartite=1, tipo="Profesor")

    # Agregar aristas al grafo dependiendo cuales profesores estan disponibles para cada materia
    for curso in cursos:
        for profesor in profesores:
            nombre_profesor = profesor['Nombre']
            #si el profesor está disponible en ese curso, se crea una arista que los conecta
            if nombre_profesor in curso["Profesores disponibles"]:
                id_curso = curso["ID"]
                grafo.add_edge(nombre_profesor, id_curso)

    return grafo

Esta función me genera los nodos y de cada conjunto y después los conecta con aristas con el parametro si dan la clase o no

In [ ]:
def nodos(grafo_bipartito):
    # esto colorea los nodos de tipo curso de color azul y los demás de rojo
    colores = ['blue' if grafo_bipartito.nodes[nodo]['tipo'] == 'Curso' else 'red' for nodo in grafo_bipartito.nodes()]

    # Dibujar el grafo coloreado
    pos = nx.spring_layout(grafo_bipartito)
    nx.draw(grafo_bipartito, pos, with_labels=True, font_weight='bold', node_color=colores, cmap=plt.cm.rainbow)
    plt.show()

# Generar el grafo bipartito
grafo_bipartito = generar_grafo_bipartito(datos)

#dibuja el grafo
nodos(grafo_bipartito)

por ultimo hice una función que itera en los datos del json para mostrar en pantalla los datos de cada profesor, esta función la haré en cada uno de los diccionarios de json

In [ ]:
#muestra todos los profesores disponibles
def mostrarprofesor():
    if 'Profesor' in datos['Datos'][0]:
        profesores = datos["Datos"][0]["Profesor"]
        print('PROFESORES ->')
        for profesor in profesores:
            print(f" ID: {profesor['id']},\n Nombre: {profesor['Nombre']}, \n Horarios disponibles: ")
            for horario in profesor['Horarios Disponibles']:
                print(f'    - {horario}')
                
        else: return print("No hay profesores en los datos.")
    else: print("No hay información sobre profesores en el archivo.")

19/11/2023

Despues lo que hice fue mover las funciones del grafo y agregar una clase llamada bipartito, esto para poder agregar nodos de profesor y curso desde esta clase y las funciones se convirtieron en métodos de la clase

In [ ]:
#Grafo bipartito
class Bipartito:
    def __init__(self,grafo):
        self.grafo = self.generar_grafo_bipartito(datos)

    #muestra todos los profesores disponibles
    def mostrarprofesores(self):
        if 'Profesor' in datos['Datos'][0]:
            profesores = datos["Datos"][0]["Profesor"]
            print('PROFESORES ->')
            for profesor in profesores:
                print(f" ID: {profesor['id']}\n Nombre: {profesor['Nombre']}\n Horarios disponibles: ")
                for horario in profesor['Horarios Disponibles']:
                    print(f'    - {horario}')
                print('')
        else: print("No hay información de profesores")

    def buscarprofe(self):
        pass

    def agregarprofe(self):
        pass

    def borrarprofe(self):
        pass

    #muestra los cursos disponibles
    def mostrarCursos(self):
        if 'Curso' in datos['Datos'][2]:
            cursos = datos['Datos'][2]['Curso']
            print('Cursos ->')
            for curso in cursos:
                print(f" ID: {curso['ID']}\n Nombre: {curso['Nombre']}\n Horas por semana -> {curso['Cantidad de horas']}\n Laboratorio: {curso['Uso de laboratorio']}")
                for profesor in curso['Profesores disponibles']:
                    print(f'    - {profesor}')
                print(f" Capacidad máxima de alumnos: {curso['Cantidad máxima de alumnos']}\n")
        else: print("No hay información de los cursos")

    #función para generar grafo bipartito
    def generar_grafo_bipartito(self, datos):
        grafo = nx.Graph()
        
        #conjunto de datos 'Cursos'
        cursos = datos["Datos"][2]["Curso"]
        #se crea un nodo por cada elemento del conjunto
        for curso in cursos:
            id_curso = curso["ID"]
            grafo.add_node(id_curso, bipartite=0, tipo="Curso")

        # Conjunto de datos 'Profesores'
        profesores = datos["Datos"][0]["Profesor"]
        #se crea un nodo por cada elemento del conjunto
        for profesor in profesores:
            nombre_profesor = profesor['Nombre']
            grafo.add_node(nombre_profesor, bipartite=1, tipo="Profesor")

        # Agregar aristas al grafo dependiendo cuales profesores estan disponibles para cada materia
        for curso in cursos:
            for profesor in profesores:
                nombre_profesor = profesor['Nombre']
                #si el profesor está disponible en ese curso, se crea una arista que los conecta
                if nombre_profesor in curso["Profesores disponibles"]:
                    id_curso = curso["ID"]
                    grafo.add_edge(nombre_profesor, id_curso)
        return grafo

    def mostrar_nodos(self):
        # esto colorea los nodos de tipo curso de color azul y los demás de rojo
        colores = ['blue' if self.grafo.nodes[nodo]['tipo'] == 'Curso' else 'red' for nodo in self.grafo.nodes()]
        # Dibujar el grafo coloreado
        pos = nx.circular_layout(self.grafo)
        nx.draw(self.grafo, pos, with_labels=True, font_weight='bold', node_color=colores)
        plt.show()

Además agregué una función main para llamar a todas las funciones

In [ ]:
if __name__ == "__main__":
    # Aquí deberías cargar tus datos desde el archivo JSON
    archivo_json = "datos2.json"
    with open(archivo_json, 'r', encoding='utf-8') as file:
        datos = json.load(file)

    grafo1 = Bipartito(datos)
    grafo1.mostrarprofesores()
    grafo1.mostrarCursos()
    grafo1.mostrar_nodos()

Por último implementé los métodos de buscar profesor, buscar curso y el de generar horarios

In [ ]:
def buscarProfe(self):
        # Obtener la lista de profesores del archivo JSON
        profesores = datos["Datos"][0]["Profesor"]

        nombre_profesor = str(input('Qué profesor buscas? -> '))

        for profesor in profesores:
            if nombre_profesor.upper() in profesor['Nombre'].upper():
                print(f" ID: {profesor['id']}\n Nombre: {profesor['Nombre']}\n Horarios disponibles: {profesor['Horarios Disponibles']}")
                return
        print(f"No se encontró ningún profesor con el nombre '{nombre_profesor}'.")
        self.buscarProfe()